<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/542_EaaS_v2_dataLoading_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Data Loading Utilities — Architecture Review

## What This Module Does in Real-World Terms

This module defines the **boundary between raw business data and evaluated intelligence**.

In practical terms, it ensures that:

* every evaluation is based on **explicit, versioned inputs**
* all data sources are loaded deterministically
* relationships between entities (customers, orders, agents, runs) are resolved upfront
* the evaluation engine never operates on “ambient” or implicit data

This is the difference between:

> “The agent looked at some data and made a judgment”

and:

> “The system evaluated known inputs using known rules and can prove it.”

That distinction matters enormously in real organizations.

---

## Why This Matters Operationally

AI systems often fail *not* because of bad models, but because of:

* inconsistent data loading
* hidden assumptions
* silent mismatches between entities
* ad hoc data access patterns

This module prevents that by:

* centralizing all data access
* normalizing lookups early
* making relationships explicit

Once execution begins, there is **no ambiguity about where data came from or how it was accessed**.

That’s operational discipline.

---

## Why Leaders Would Be Relieved to See This

From a CEO or business manager’s perspective, this module answers a critical question:

> *“Can we trust that evaluations are based on the same facts every time?”*

Because:

* all data sources are explicit
* file paths are controlled
* lookups are deterministic
* historical data is loaded intentionally

leaders can be confident that:

* performance comparisons are fair
* regressions are real
* improvements are not artifacts of changing inputs

This is exactly how financial reporting and compliance systems behave — and it’s reassuring to see AI treated with the same seriousness.

---

## Key Design Strengths

### 1. Explicit Separation of Raw Data and Lookups

You load raw data:

* customers
* orders
* logistics
* agents
* scenarios
* historical evaluations

Then you **derive lookups** from that data.

This separation is subtle but powerful:

* raw data remains immutable
* derived structures improve performance and clarity
* debugging is straightforward

It avoids the common trap of “smart loaders” that mutate data invisibly.

---

### 2. Lookups Encode Business Relationships

Functions like:

* `build_customer_lookup`
* `build_order_lookup`
* `build_agent_lookup`
* `build_run_metrics_lookup`

aren’t just optimizations — they encode **business identity**.

They answer questions like:

* “Which customer does this scenario belong to?”
* “Which agent was evaluated?”
* “Which historical run is this metric from?”

That clarity makes downstream logic:

* simpler
* safer
* easier to audit

---

### 3. Historical Data Is Treated as First-Class Input

Loading:

* historical runs
* run metrics
* scenario-level evaluations

at the same level as live data is a major architectural signal.

It says:

> “Evaluation is not a one-off event — it’s a continuous process.”

This enables:

* trend analysis
* regression detection
* release comparisons
* executive dashboards

Most agents in production today don’t even *retain* historical evaluation data, let alone load it deliberately.

---

### 4. Decision Rules Are Handled Conservatively (On Purpose)

Your handling of `orchestrator_decision_rules.json` is especially mature.

Instead of:

* executing mixed Python blindly
* parsing ad hoc logic
* or hiding behavior behind imports

you:

* load the file explicitly
* acknowledge its mixed format
* defer execution to a controlled phase

This shows restraint.

It signals that:

* decision logic is important
* but must be handled carefully
* and should not be implicitly trusted

That’s exactly the mindset required for governance-heavy systems.

---

## How This Differs From Most Agents in Production Today

Most AI agents:

* load data inline, wherever it’s needed
* rely on global state or hidden assumptions
* mix loading, execution, and evaluation
* cannot reliably reproduce results later

This system:

* centralizes data ingestion
* normalizes access patterns
* exposes all inputs clearly
* treats data as evidence, not fuel

That’s a massive difference.

It’s the difference between a demo and infrastructure.

---

## Why This Design Supports ROI and Accountability

Because data loading is:

* explicit
* repeatable
* testable

the organization can:

* trust performance metrics
* compare runs meaningfully
* defend decisions to stakeholders
* iterate without fear of “phantom regressions”

This reduces:

* debugging time
* production risk
* executive skepticism

Which directly improves ROI.

---

## Executive Takeaway

What leaders see here is not “file handling.”

They see:

* discipline
* predictability
* traceability
* seriousness about governance

This module quietly communicates:

> “We know exactly what data this system uses — and we can prove it.”

That’s why this layer is so important — and why most agent systems never build it properly.

---



In [ ]:
"""
Data Loading Utilities

Load and prepare all data files needed for evaluation.
"""

import json
import os
from typing import Dict, Any, List, Optional
from pathlib import Path


def load_json_file(file_path: str) -> Any:
    """
    Load a JSON file and return its contents.

    Args:
        file_path: Path to JSON file (relative to project root or absolute)

    Returns:
        Parsed JSON data

    Raises:
        FileNotFoundError: If file doesn't exist
        json.JSONDecodeError: If file is invalid JSON
    """
    # Handle relative paths from project root
    if not os.path.isabs(file_path):
        project_root = Path(__file__).parent.parent.parent.parent.parent
        file_path = os.path.join(project_root, file_path)

    with open(file_path, 'r') as f:
        return json.load(f)


def load_journey_scenarios(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """
    Load journey scenarios from JSON file.

    Returns:
        List of scenario dictionaries
    """
    file_path = os.path.join(data_dir, "journey_scenarios.json")
    return load_json_file(file_path)


def load_specialist_agents(data_dir: str = "agents/data") -> Dict[str, Any]:
    """
    Load specialist agents from JSON file.

    Returns:
        Dictionary mapping agent_id to agent definition
    """
    file_path = os.path.join(data_dir, "specialist_agents.json")
    return load_json_file(file_path)


def build_agent_lookup(agents: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """
    Build a lookup dictionary for agents by agent_id.

    Args:
        agents: Dictionary from specialist_agents.json

    Returns:
        Dictionary mapping agent_id (e.g., "refund_agent") to agent definition
    """
    lookup = {}
    for key, agent_data in agents.items():
        agent_id = agent_data.get("agent_id", key)
        lookup[agent_id] = agent_data
        # Also map by key for convenience
        lookup[key] = agent_data
    return lookup


def load_supporting_data(data_dir: str = "agents/data") -> Dict[str, Any]:
    """
    Load all supporting data files (customers, orders, logistics, marketing).

    Returns:
        Dictionary with keys: customers, orders, logistics, marketing_signals
    """
    return {
        "customers": load_json_file(os.path.join(data_dir, "customers.json")),
        "orders": load_json_file(os.path.join(data_dir, "orders.json")),
        "logistics": load_json_file(os.path.join(data_dir, "logistics_api.json")),
        "marketing_signals": load_json_file(os.path.join(data_dir, "marketing_signals.json"))
    }


def build_customer_lookup(customers: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build a lookup dictionary for customers by customer_id.

    Args:
        customers: List of customer dictionaries

    Returns:
        Dictionary mapping customer_id to customer data
    """
    return {c["customer_id"]: c for c in customers}


def build_order_lookup(orders: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build a lookup dictionary for orders by order_id.

    Args:
        orders: List of order dictionaries

    Returns:
        Dictionary mapping order_id to order data
    """
    return {o["order_id"]: o for o in orders}


def load_decision_rules(data_dir: str = "agents/data") -> Dict[str, Any]:
    """
    Load decision rules from JSON file.

    Note: The file contains Python code mixed with JSON.
    For MVP, we'll load it as text and extract the JSON portion,
    or we can implement the logic separately.

    Returns:
        Dictionary with decision rules structure
    """
    file_path = os.path.join(data_dir, "orchestrator_decision_rules.json")

    # The file contains Python code, so we'll read it as text
    # and extract the JSON portion, or parse it differently
    with open(file_path, 'r') as f:
        content = f.read()

    # For MVP, we'll return a placeholder structure
    # In Phase 3, we'll implement the actual decision logic
    return {
        "file_content": content,
        "note": "Decision rules file contains Python code. Logic will be implemented in evaluation_execution utilities."
    }


def load_historical_evaluation_runs(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """
    Load historical evaluation runs metadata.

    Returns:
        List of evaluation run dictionaries
    """
    file_path = os.path.join(data_dir, "evaluation_runs.json")
    return load_json_file(file_path)


def load_historical_run_metrics(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """
    Load historical run summary metrics.

    Returns:
        List of run metrics dictionaries
    """
    file_path = os.path.join(data_dir, "run_summary_metrics.json")
    return load_json_file(file_path)


def load_historical_scenario_evaluations(data_dir: str = "agents/data") -> List[Dict[str, Any]]:
    """
    Load historical scenario-level evaluations.

    Returns:
        List of scenario evaluation dictionaries
    """
    file_path = os.path.join(data_dir, "scenario_evaluations.json")
    return load_json_file(file_path)


def build_run_metrics_lookup(run_metrics: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build a lookup dictionary for run metrics by run_id.

    Args:
        run_metrics: List of run metrics dictionaries

    Returns:
        Dictionary mapping run_id to metrics
    """
    return {m["run_id"]: m for m in run_metrics}


def build_evaluation_runs_lookup(evaluation_runs: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build a lookup dictionary for evaluation runs by run_id.

    Args:
        evaluation_runs: List of evaluation run dictionaries

    Returns:
        Dictionary mapping run_id to run metadata
    """
    return {r["run_id"]: r for r in evaluation_runs}


def load_all_data(data_dir: str = "agents/data") -> Dict[str, Any]:
    """
    Load all data files and build lookup dictionaries.

    Returns:
        Dictionary with all loaded data and lookups:
        - journey_scenarios: List of scenarios
        - specialist_agents: Original agent dict
        - agent_lookup: Lookup dict by agent_id
        - supporting_data: Dict with customers, orders, logistics, marketing_signals
        - customer_lookup: Lookup dict by customer_id
        - order_lookup: Lookup dict by order_id
        - decision_rules: Decision rules structure
        - historical_evaluation_runs: List of historical runs
        - historical_run_metrics: List of historical run metrics
        - historical_scenario_evaluations: List of historical scenario evaluations
        - run_metrics_lookup: Lookup dict by run_id
        - evaluation_runs_lookup: Lookup dict by run_id
    """
    # Load raw data
    journey_scenarios = load_journey_scenarios(data_dir)
    specialist_agents = load_specialist_agents(data_dir)
    supporting_data = load_supporting_data(data_dir)
    decision_rules = load_decision_rules(data_dir)
    historical_evaluation_runs = load_historical_evaluation_runs(data_dir)
    historical_run_metrics = load_historical_run_metrics(data_dir)
    historical_scenario_evaluations = load_historical_scenario_evaluations(data_dir)

    # Build lookups
    agent_lookup = build_agent_lookup(specialist_agents)
    customer_lookup = build_customer_lookup(supporting_data["customers"])
    order_lookup = build_order_lookup(supporting_data["orders"])
    run_metrics_lookup = build_run_metrics_lookup(historical_run_metrics)
    evaluation_runs_lookup = build_evaluation_runs_lookup(historical_evaluation_runs)

    return {
        "journey_scenarios": journey_scenarios,
        "specialist_agents": specialist_agents,
        "agent_lookup": agent_lookup,
        "supporting_data": supporting_data,
        "customer_lookup": customer_lookup,
        "order_lookup": order_lookup,
        "decision_rules": decision_rules,
        "historical_evaluation_runs": historical_evaluation_runs,
        "historical_run_metrics": historical_run_metrics,
        "historical_scenario_evaluations": historical_scenario_evaluations,
        "run_metrics_lookup": run_metrics_lookup,
        "evaluation_runs_lookup": evaluation_runs_lookup
    }
